In [ ]:
from ortools.sat.python import cp_model
from datetime import datetime

def schedule_rooms_ortools(rooms, requests, floor_gap_weight=0.7, capacity_weight=0.3):
    model = cp_model.CpModel()

    # Variables: Define a boolean variable for each room-request-time_slot combination
    room_vars = {}
    for r_id, room in enumerate(rooms):
        for req_id, request in enumerate(requests):
            for t_id, time_slot in enumerate(request["time_slots"]):
                room_vars[(r_id, req_id, t_id)] = model.NewBoolVar(f'room_{r_id}_req_{req_id}_time_{t_id}')

    # Constraints: Ensure each request is assigned to exactly one room (if possible)
    # For each request, find the valid rooms and time slots
    for req_id, request in enumerate(requests):
        num_people = request["num_people"]
        request_type = request["type"]
        time_slots = request["time_slots"]

        # Initialize empty lists before the loop starts
        valid_rooms_for_request = []  # List to hold valid room variables for the current request
        valid_room_names = []  # List to store the names of valid rooms for debugging

        # Check each room for the current request
        for r_id, room in enumerate(rooms):
            for t_id, time_slot in enumerate(time_slots):
                # Check if room has sufficient capacity for the request type
                if request_type in room["type_capacity"] and room["type_capacity"][request_type] >= num_people:
                    valid_rooms_for_request.append(room_vars[(r_id, req_id, t_id)])
                    valid_room_names.append(room["name"])

        # Print the available rooms for the current request
        print(f"Available rooms for request {req_id} ({request_type}) with {num_people} people: {valid_room_names}")
        print(valid_rooms_for_request)

    # If there are valid rooms, ensure exactly one room is assigned
    if valid_rooms_for_request:
        model.Add(sum(valid_rooms_for_request) == 1)
    else:
        print(f"Warning: Request {req_id} cannot be fulfilled with the available rooms!")

    # Room assignment limit: A room can only be assigned to one request for a given time slot
    for r_id, room in enumerate(rooms):
        for t_id in range(len(requests[0]["time_slots"])):  # Loop through time slots for each room
            # Ensure that a room can be assigned to at most one request for a given time slot
            room_assignments = [room_vars[(r_id, req_id, t_id)] for req_id in range(len(requests))]
            model.Add(sum(room_assignments) <= 1)

    # Time slot constraints to prevent overlaps within the same request and across different requests
    for r_id, room in enumerate(rooms):
        for req_id1, request1 in enumerate(requests):
            for t_id1, time_slot1 in enumerate(request1["time_slots"]):
                start1, end1 = [datetime.strptime(ts, "%Y-%m-%d %H:%M") for ts in time_slot1]
                for req_id2, request2 in enumerate(requests):
                    for t_id2, time_slot2 in enumerate(request2["time_slots"]):
                        if req_id1 != req_id2 or t_id1 != t_id2:
                            start2, end2 = [datetime.strptime(ts, "%Y-%m-%d %H:%M") for ts in time_slot2]
                            if not (end1 <= start2 or end2 <= start1):  # Time slots overlap
                                model.AddBoolOr([
                                    room_vars[(r_id, req_id1, t_id1)].Not(),
                                    room_vars[(r_id, req_id2, t_id2)].Not()
                                ])

    # Objective function: (Optional) minimize the gap between floor assignments and unused capacity
    floors_used = []
    for r_id, room in enumerate(rooms):
        for req_id, request in enumerate(requests):
            for t_id, time_slot in enumerate(request["time_slots"]):
                if request["type"] in room["type_capacity"]:
                    floors_used.append(room_vars[(r_id, req_id, t_id)] * room["floor"])

    global_min_floor = model.NewIntVar(-100, 100, "global_min_floor")
    global_max_floor = model.NewIntVar(-100, 100, "global_max_floor")
    model.AddMinEquality(global_min_floor, floors_used)
    model.AddMaxEquality(global_max_floor, floors_used)
    global_floor_gap = global_max_floor - global_min_floor

    # Unused capacity calculation (minimizing unused space)
    unused_capacity = []
    for r_id, room in enumerate(rooms):
        for req_id, request in enumerate(requests):
            for t_id, time_slot in enumerate(request["time_slots"]):
                if request["type"] in room["type_capacity"]:
                    unused_capacity.append(room_vars[(r_id, req_id, t_id)] * (room["type_capacity"][request["type"]] - request["num_people"]))

    # Optional: minimize the unused capacity
    model.Minimize(floor_gap_weight * global_floor_gap + capacity_weight * sum(unused_capacity))

    # Solve the model
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # Check if a feasible solution is found
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print('Solution:')
        for r_id, room in enumerate(rooms):
            for req_id, request in enumerate(requests):
                for t_id, time_slot in enumerate(request["time_slots"]):
                    if solver.BooleanValue(room_vars[(r_id, req_id, t_id)]):
                        print(f'Room {room["name"]} assigned to request {req_id} for time slot {time_slot}')
    else:
        print('No solution found.')

    # Debugging: Print the values of room_vars
    for r_id, room in enumerate(rooms):
        for req_id, request in enumerate(requests):
            for t_id, time_slot in enumerate(request["time_slots"]):
                print(f'room_vars[{r_id}, {req_id}, {t_id}] = {solver.BooleanValue(room_vars[(r_id, req_id, t_id)])}')

# Example usage:
rooms = [
    {"name": "LES VOÛTES", "type_capacity": {"Theatre": 80, "Reception": 230}, "floor": 0},
    {"name": "ARCHAMBAULT", "type_capacity": {"Theatre": 60, "Reception": 50}, "floor": 0},
    {"name": "LA GALERIE D’ART", "type_capacity": {"Theatre": 100, "Reception": 50}, "floor": 1},
    {"name": "CHEZ PLUME", "type_capacity": {"Theatre": 0, "Reception": 100}, "floor": 1},
    {"name": "S. BERNHARDT ", "type_capacity": {"Theatre": 220, "Reception": 320}, "floor": 2},
]
requests = [
    {"type": "Theatre", "num_rooms": 1, "num_people": 80, "time_slots": [["2023-10-01 13:00", "2023-10-01 15:00"]]},
    {"type": "Theatre", "num_rooms": 1, "num_people": 80, "time_slots": [["2023-10-01 14:00", "2023-10-01 15:00"]]},
    {"type": "Reception", "num_rooms": 1, "num_people": 200, "time_slots": [["2023-10-01 14:45", "2023-10-01 19:00"]]},
]
schedule_rooms_ortools(rooms, requests, floor_gap_weight=0.7, capacity_weight=0.3)


Available rooms for request 0 (Theatre) with 80 people: ['LES VOÛTES', 'LA GALERIE D’ART', 'S. BERNHARDT ']
Available rooms for request 1 (Theatre) with 80 people: ['LES VOÛTES', 'LA GALERIE D’ART', 'S. BERNHARDT ']
Available rooms for request 2 (Reception) with 200 people: ['LES VOÛTES', 'S. BERNHARDT ']
Solution:
Room LES VOÛTES assigned to request 2 for time slot ['2023-10-01 14:45', '2023-10-01 19:00']
Room ARCHAMBAULT assigned to request 2 for time slot ['2023-10-01 14:45', '2023-10-01 19:00']
Room LA GALERIE D’ART assigned to request 2 for time slot ['2023-10-01 14:45', '2023-10-01 19:00']
Room CHEZ PLUME assigned to request 2 for time slot ['2023-10-01 14:45', '2023-10-01 19:00']
room_vars[0, 0, 0] = False
room_vars[0, 1, 0] = False
room_vars[0, 2, 0] = True
room_vars[1, 0, 0] = False
room_vars[1, 1, 0] = False
room_vars[1, 2, 0] = True
room_vars[2, 0, 0] = False
room_vars[2, 1, 0] = False
room_vars[2, 2, 0] = True
room_vars[3, 0, 0] = False
room_vars[3, 1, 0] = False
room_vars[

NameError: name 'valid_rooms_for_request' is not defined

In [57]:
from ortools.linear_solver import pywraplp
from datetime import datetime

def assign_rooms_to_requests(rooms, requests, floor_weight=1, space_weight=1):
    # Helper function to check if two time slots overlap
    def timeslot_overlap(ts1, ts2):
        # Convert string time slots to datetime objects
        start1, end1 = [datetime.strptime(ts, "%Y-%m-%d %H:%M") for ts in ts1]
        start2, end2 = [datetime.strptime(ts, "%Y-%m-%d %H:%M") for ts in ts2]
        
        # Check if there is an overlap
        return not (end1 <= start2 or end2 <= start1)

    # Initialize the solver
    solver = pywraplp.Solver.CreateSolver('SCIP')

    # Room assignment variables
    room_vars = {}

    # Define valid rooms for each request
    valid_rooms_for_request = {}

    # Define room assignment variables based on request and time slot overlap
    for r_id, room in enumerate(rooms):
        for req_id, request in enumerate(requests):
            for t_id, time_slot in enumerate(request['time_slots']):
                # Check if room meets request's capacity
                if request["type"] in room["type_capacity"] and room["type_capacity"][request["type"]] >= request["num_people"]:
                    # Check for time slot overlaps
                    valid_assignment = True
                    for existing_req_id, existing_request in enumerate(requests):
                        if existing_req_id != req_id:  # Skip the current request
                            for existing_t_id, existing_time_slot in enumerate(existing_request['time_slots']):
                                if timeslot_overlap(time_slot, existing_time_slot):
                                    # If there is a time overlap, invalidate the room assignment for this time slot
                                    if (r_id, existing_req_id, existing_t_id) in room_vars and room_vars[(r_id, existing_req_id, existing_t_id)].solution_value() == 1:
                                        valid_assignment = False
                                        break
                    if valid_assignment:
                        # Define the room assignment variable for this request, room, and time slot
                        room_vars[(r_id, req_id, t_id)] = solver.BoolVar(f"room_{r_id}_req_{req_id}_time_{t_id}")
                        
                        # Collect valid rooms for the current request (based on capacity and time slot)
                        if req_id not in valid_rooms_for_request:
                            valid_rooms_for_request[req_id] = []
                        valid_rooms_for_request[req_id].append(room_vars[(r_id, req_id, t_id)])

    # Debugging: Print the valid rooms for each request
    for req_id in valid_rooms_for_request:
        valid_room_names = []
        for room_var in valid_rooms_for_request[req_id]:
            r_id, req_id, t_id = room_var.name().split('_')[1], room_var.name().split('_')[3], room_var.name().split('_')[5]
            valid_room_names.append(rooms[int(r_id)]["name"])  # Get room name based on id
        print(f"Available rooms for request {req_id}: {valid_room_names}")

    # Add constraint: each request can only be assigned one room at a time
    for req_id, valid_rooms in valid_rooms_for_request.items():
        solver.Add(solver.Sum(valid_rooms) == 1)

    # Add constraint: each room can only be assigned to one request at a time
    for r_id, room in enumerate(rooms):
        for t_id, time_slot in enumerate(requests[0]['time_slots']):  # Arbitrarily pick the first request for checking time slots
            valid_rooms_for_this_time = []
            for req_id, request in enumerate(requests):
                if request["type"] in room["type_capacity"] and room["type_capacity"][request["type"]] >= request["num_people"]:
                    valid_rooms_for_this_time.append(room_vars.get((r_id, req_id, t_id)))
            solver.Add(solver.Sum(valid_rooms_for_this_time) <= 1)  # Ensuring that a room is not assigned to more than one request at any time

    # Add an objective function to minimize the floor distance and unused space
    total_distance = solver.IntVar(0, solver.infinity(), "total_distance")
    total_unused_space = solver.IntVar(0, solver.infinity(), "total_unused_space")
    solver.Minimize(total_distance + total_unused_space)

    # Define the floor distance and unused space terms
    distance_terms = []
    unused_space_terms = []

    for r_id, room in enumerate(rooms):
        for req_id, request in enumerate(requests):
            for t_id, time_slot in enumerate(request['time_slots']):
                if (r_id, req_id, t_id) in room_vars:
                    # Calculate the floor distance as absolute difference between floors
                    floor_distance = abs(rooms[r_id]["floor"] - rooms[req_id]["floor"])  # Minimize based on request floor
                    distance_terms.append(room_vars[(r_id, req_id, t_id)] * floor_distance * floor_weight)

                    # Calculate unused space (room capacity - number of people)
                    unused_space = room["type_capacity"][request["type"]] - request["num_people"]
                    unused_space_terms.append(room_vars[(r_id, req_id, t_id)] * unused_space * space_weight)

    # Assign weighted total distance and unused space
    solver.Add(total_distance == solver.Sum(distance_terms))
    solver.Add(total_unused_space == solver.Sum(unused_space_terms))

    # Debug: Print the objective function terms before solving
    print("Objective function terms:")
    print("Floor distance terms:", distance_terms)
    print("Unused space terms:", unused_space_terms)

    # Solve the problem
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print("Solution found:")
        for r_id, req_id, t_id in room_vars:
            if room_vars[(r_id, req_id, t_id)].solution_value() == 1:
                print(f"Request {req_id} assigned to Room {rooms[r_id]['name']} for time slot {requests[req_id]['time_slots'][t_id]}")
    else:
        print("No solution found.")

# Example usage
rooms = [
    {"name": "LES VOÛTES", "type_capacity": {"Theatre": 80, "Reception": 230}, "floor": 0},
    {"name": "ARCHAMBAULT", "type_capacity": {"Theatre": 60, "Reception": 50}, "floor": 0},
    {"name": "LA GALERIE D’ART", "type_capacity": {"Theatre": 100, "Reception": 50}, "floor": 1},
    {"name": "CHEZ PLUME", "type_capacity": {"Theatre": 0, "Reception": 100}, "floor": 1},
    {"name": "S. BERNHARDT ", "type_capacity": {"Theatre": 220, "Reception": 320}, "floor": 2},
]

requests = [
    {"type": "Theatre", "num_rooms": 1, "num_people": 80, "time_slots": [["2023-10-01 13:00", "2023-10-01 15:00"]]},
    {"type": "Theatre", "num_rooms": 1, "num_people": 80, "time_slots": [["2023-10-01 16:00", "2023-10-01 17:00"]]},
    {"type": "Reception", "num_rooms": 1, "num_people": 200, "time_slots": [["2023-10-01 15:05", "2023-10-01 19:00"]]},
]

# Call the function to solve the problem
assign_rooms_to_requests(rooms, requests, floor_weight=1, space_weight=0.5)


Available rooms for request 0: ['LES VOÛTES', 'LA GALERIE D’ART', 'S. BERNHARDT ']
Available rooms for request 1: ['LES VOÛTES', 'LA GALERIE D’ART', 'S. BERNHARDT ']
Available rooms for request 2: ['LES VOÛTES', 'S. BERNHARDT ']
Objective function terms:
Floor distance terms: [<ortools.linear_solver.python.linear_solver_natural_api.ProductCst object at 0x000001CAD41DB350>, <ortools.linear_solver.python.linear_solver_natural_api.ProductCst object at 0x000001CAD49CF090>, <ortools.linear_solver.python.linear_solver_natural_api.ProductCst object at 0x000001CAD4875D10>, <ortools.linear_solver.python.linear_solver_natural_api.ProductCst object at 0x000001CAD4874590>, <ortools.linear_solver.python.linear_solver_natural_api.ProductCst object at 0x000001CAD4874D10>, <ortools.linear_solver.python.linear_solver_natural_api.ProductCst object at 0x000001CAD4875210>, <ortools.linear_solver.python.linear_solver_natural_api.ProductCst object at 0x000001CAD4C13D90>, <ortools.linear_solver.python.linear